In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

(train_set, test_set), dataset_info = tfds.load('cats_vs_dogs', split=['train[:81%]', 'train[80%:]'], as_supervised=True, with_info=True)

In [2]:
#Get only cat Image

def preprocess(img, label):
    img = tf.image.resize(img, (150, 150))
    img = img / 255.0  # Normalize to [0,1]
    return img, label

train_set = train_set.map(preprocess).batch(32).shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [3]:
cats_dataset, dogs_dataset = [], []
for images, labels in train_set.take(3):
    for i in range(32):
        if labels[i] == 0:
            cats_dataset.append(tf.expand_dims(images[i], axis=0))
        else:
            dogs_dataset.append(tf.expand_dims(images[i], axis=0))
            

In [4]:
#Load model 
classifier = tf.keras.models.load_model('cats_dogs_classifier.h5')

In [5]:
import numpy as np 

for i in cats_dataset:
    print(np.argmax(classifier.predict(i), axis=1), end=" ")
    
print(train_set)

[0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [1] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] <PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [6]:
#left and right hemi
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate

left_input = Input(shape=(150, 150, 3))
left_flatten = Flatten()(left_input)
left_hemi = Dense(512)(left_flatten)
left_hemi = Dense(256)(left_hemi)

right_input = Input(shape=(150, 150, 3))
right_flatten = Flatten()(right_input)
right_hemi = Dense(512)(right_flatten)
right_hemi = Dense(256)(right_hemi)

In [7]:
#Corpus Callosum equivalence

left_right_nn = Dense(512)(left_hemi)
right_left_nn = Dense(512)(right_hemi)

left_hemi = Concatenate(axis=1)([left_hemi, right_left_nn])
right_hemi = Concatenate(axis=1)([right_hemi, left_right_nn])

central_input = Concatenate(axis=1)([left_hemi, right_hemi])
central_layer = Dense(1024)(central_input)
central_layer = Dense(512)(central_layer)
central_layer = Dense(128)(central_layer)
central_output = Dense(2)(central_layer)


In [8]:
train_left_input, train_right_input = [], []
label_left_input, label_right_input = [], []
for batch in train_set:
    train_left_input.append(batch[0])
    train_right_input.append(batch[0])
    label_left_input.append(batch[1])
    label_right_input.append(batch[1])
    

In [9]:
model = Model(inputs=[left_input, right_input], outputs=[left_input, right_input])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics="accuracy")

model.fit([train_left_input, train_right_input], [label_left_input, label_right_input], epochs=3, verbose=1, validation_data=test_set)

ValueError: Data cardinality is ambiguous:
  x sizes: 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32

589